# Function Calling Use case with Google Gemini model

## Lab Description:

This lab demonstrates how to integrate function calling capabilities with the Google Gemini model. It begins by setting up the Google API key and defining a simple arithmetic function. The function is then passed as a tool during LLM initialization, enabling the model to invoke it dynamically. The lab concludes by exploring parallel function calling features of the Gemini model.

## Lab Objectives:

After completing the lab, the participants will be able to:

- Set up and authenticate the Google Gemini API for function calling.
- Define and integrate custom functions as tools within the Gemini model.
- Utilize function calling during LLM execution to enhance model capabilities.
- Explore and implement parallel function calling to handle multiple tasks efficiently.

## What is function calling ?


Function calling gfeature in the Gemini API allows the model to generate structured outputs that specify function names and arguments. The model itself doesn't invoke the functions directly but provides the necessary information to call external APIs. After the API is called using the suggested function and arguments, the output can be used to further query the model, enabling it to give more comprehensive responses.

## How function calling works ? 

A structured query data describing the function is passed to a model prompt. It includes all the details like the function name, parameters, description of the function and parameters and so on. The model then analyzes this description to effectively provide responses. 

## Import the packages

In [2]:
import google.generativeai as genai


/home/ubuntu/jupyter/gemini_function_calling/gemini/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



## Generate a Google Studio API Key

We will create an API key that provides secure access to the Gemini API, enabling the Jupyter Notebook to communicate with the servers.

### Set Up an AI Studio Account

1. Visit **AI Studio** and click on **"Sign in to AI Studio"**.  
   
      <img src="one.png" alt="Figure 1: Sign in page of AI Studio" width="600" height="400">

### Obtain Your API Key

2. After logging into AI Studio, scroll down and select **"Get your API Key."**  
   
      <img src="two.png" alt="Figure 1: Sign in page of AI Studio" width="600" height="400">

### Create the API Key

3. Click on **"Get API Key"**, then select **"Create API Key"**, and then choose **"Create API key in new project"**.  
   
   <img src="three.png" alt="Figure 1: Sign in page of AI Studio" width="600" height="400">

   <img src="four.png" alt="Figure 1: Sign in page of AI Studio" width="600" height="400">

### Copy the API Key

4. After generating the key, **copy** it for use in your Jupyter Notebook.  

   <img src="five.png" alt="Figure 1: Sign in page of AI Studio" width="600" height="400">


## Setting the GOOGLE API KEY

You can create tour API KEY via https://aistudio.google.com. Once your key is generated, it is required to set it as environment variable. It needs to be activated by passing it into genai.configure()

In [3]:
import os

GOOGLE_API_KEY = input("Please enter your Google API Key: ")
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

genai.configure(api_key=GOOGLE_API_KEY) 

## Define a function 

Let us first define a really simple function to understad and test function calling. 

In [8]:
def add(a:float, b:float):
    """returns a + b."""
    return a + b


This is a simple function that adds two numbers. Let us see if we could make the model use this function to answer our query. To use the function calling feature, the function should be declared at the model initialization itself.

## Declare function during model initialization

In [4]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash', tools=[add]) #The tools param is used to declare functions.

We can now use the model to use the defined function to answer our query. 

## Generate function call

In [5]:
chat = model.start_chat(enable_automatic_function_calling=True) #enable_automatic_function_calling should be set to true to have the SDK automatically use the function 
response = chat.send_message('What is 57 added to 22 ?')
response.text

'57 added to 22 is 79.'

The model automatically uses the function to find answer to our query. Let us now see what the flow is like to confirm if the model is actually using the function and not just generating answer on it's own. 

chat.history contains the sequence of these events. 

In [6]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('x'*80)

user -> {'text': 'What is 57 added to 22 ?'}
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
model -> {'function_call': {'name': 'add', 'args': {'a': 57.0, 'b': 22.0}}}
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
user -> {'function_response': {'name': 'add', 'response': {'result': 79.0}}}
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
model -> {'text': '57 added to 22 is 79.'}
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


We can see that after processing user query, the model calls the function and identifies the parameters. It then executes the function locally to generate a response. 

This proves that function calling is actually working.

## Parallel Function Calling

In addition to normal function calling, you can also call multiple functions in a single turn. 

## Function Definitions

In [8]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

## Declare the functions during model initialization

Define tools: The house_fns list contains a set of functions that might be used to control a party atmosphere—specifically, power_disco_ball, start_music, and dim_lights.

Initialize model: The GenerativeModel is initialized with the model name "gemini-1.5-flash", and the previously defined tools (house_fns) are passed in as available functions for this model to use.

Start chat session: A chat session is started with the start_chat() method of the model, creating a session where the model can process and respond to messages.

Send message: The message "Turn this place into a party!" is sent to the model via the send_message method. The model generates a response based on the message and decides which of the available tools should be called.

Function extraction and printing: The response from the model is examined part by part. If any function calls are present (i.e., the model decides to use one of the tools), the function name and its arguments are extracted and printed in a readable format, showing which functions the model called and with what arguments.

In [9]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


Each of this response reflects a single function call. To send the resoonses back, it should be simulated in the same order. 

## Simulating and building the response

In [10]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

I've turned on the disco ball, started playing some energetic music at 120 bpm, and dimmed the lights. Let's get this party started! 



<div style="text-align: left;">
    <img src="logo.png" alt="flow" width="150" height="100">
</div>